In [12]:
import os
import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, transforms

# Установите устройство
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    
    in_channels = 3 
    for i in range(n_layers):
        out_channels = trial.suggest_int("n_units_l{}".format(i), 16, 128)
        kernel_size = trial.suggest_int("kernel_size", 3, 7)
        
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size//2))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_channels = out_channels
        
    layers.append(nn.Flatten())
    layers.append(nn.Linear(in_channels * 32 * 32, CLASSES))  # 32x32 изображения
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

def get_cifar10():
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

def objective(trial):
    model = define_model(trial).to(DEVICE)

    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = get_cifar10()

    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Статистика исследования:")
    print("  Количество завершенных испытаний: ", len(study.trials))
    print("  Количество обрезанных испытаний: ", len(pruned_trials))
    print("  Количество завершенных испытаний: ", len(complete_trials))

    print("Лучшее испытание:")
    trial = study.best_trial

    print("  Значение (accuracy): ", trial.value)

    print("  Параметры: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("\nЗавершенные испытания:")
    for trial in complete_trials:
        print("  Trial #{}: Value: {}, Params: {}".format(trial.number, trial.value, trial.params))

    print("\nОбрезанные испытания:")
    for trial in pruned_trials:
        print("  Trial #{}: Value: {}".format(trial.number, trial.value))

[I 2024-12-01 22:59:23,912] A new study created in memory with name: no-name-e98cdc4b-7e7e-41f6-9909-24c4d9a63dd4


100%|████████████████████████| 170498071/170498071 [02:14<00:00, 1266400.23it/s]


Extracting /Users/anna_vilkul/Documents/учеба/cifar-10-python.tar.gz to /Users/anna_vilkul/Documents/учеба


[I 2024-12-01 23:09:14,777] Trial 0 finished with value: 0.34921875 and parameters: {'n_layers': 2, 'n_units_l0': 96, 'kernel_size': 7, 'dropout_l0': 0.23726362588785577, 'n_units_l1': 54, 'dropout_l1': 0.4122368302125995, 'optimizer': 'RMSprop', 'lr': 0.0007746627515756406}. Best is trial 0 with value: 0.34921875.


Files already downloaded and verified


[I 2024-12-01 23:09:38,369] Trial 1 finished with value: 0.29140625 and parameters: {'n_layers': 1, 'n_units_l0': 42, 'kernel_size': 3, 'dropout_l0': 0.48387811806710523, 'optimizer': 'SGD', 'lr': 0.0010331697007036126}. Best is trial 0 with value: 0.34921875.


Статистика исследования:
  Количество завершенных испытаний:  2
  Количество обрезанных испытаний:  0
  Количество завершенных испытаний:  2
Лучшее испытание:
  Значение (accuracy):  0.34921875
  Параметры: 
    n_layers: 2
    n_units_l0: 96
    kernel_size: 7
    dropout_l0: 0.23726362588785577
    n_units_l1: 54
    dropout_l1: 0.4122368302125995
    optimizer: RMSprop
    lr: 0.0007746627515756406

Завершенные испытания:
  Trial #0: Value: 0.34921875, Params: {'n_layers': 2, 'n_units_l0': 96, 'kernel_size': 7, 'dropout_l0': 0.23726362588785577, 'n_units_l1': 54, 'dropout_l1': 0.4122368302125995, 'optimizer': 'RMSprop', 'lr': 0.0007746627515756406}
  Trial #1: Value: 0.29140625, Params: {'n_layers': 1, 'n_units_l0': 42, 'kernel_size': 3, 'dropout_l0': 0.48387811806710523, 'optimizer': 'SGD', 'lr': 0.0010331697007036126}

Обрезанные испытания:
